# Imports

In [1]:
import pandas
import configparser as ConfigParser
import os
import numpy as np
%load_ext autoreload
%autoreload 2
from bitfusion.graph_plot.barchart import BarChart

%matplotlib inline
import matplotlib

import warnings
warnings.filterwarnings('ignore')

import bitfusion.src.benchmarks.benchmarks as benchmarks
from bitfusion.src.simulator.stats import Stats
from bitfusion.src.simulator.simulator import Simulator
from bitfusion.src.sweep.sweep import SimulatorSweep, check_pandas_or_run
from bitfusion.src.utils.utils import *
from bitfusion.src.optimizer.optimizer import optimize_for_order, get_stats_fast

import dnnweaver2

## Constants
### We use a batch size of 16

In [2]:
# ant配置
# batch_size = 64
batch_size = 1

# 默认配置
# batch_size = 16
# batch_size = 32

results_dir = './results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

fig_dir = './fig'
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

### Create a simulator object using the configuration from BitFusion-Eyeriss (bf_e_conf.ini)

In [3]:
# 初始化+能耗仿真
# # BitFusion configuration file
# 原始样例
# config_file = 'bf_e_conf.ini'
# 测试样例
config_file = 'bf_test_conf.ini'

# ANT configuration file
# config_file = 'conf_ant.ini'

# Create simulator object
verbose = False
# verbose 调试模式启动
# verbose = True


bf_e_sim = Simulator(config_file, verbose)

# 得把这个解决
bf_e_energy_costs = bf_e_sim.get_energy_cost()

print(bf_e_sim)

energy_tuple = bf_e_energy_costs
print('')
print('*'*50)
print(energy_tuple)




{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict {'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict_obuf {'size (bytes)': 8192.0, 'block size (bytes)': 4.0, 'read-write port': 0}

NM_MAX_MIN_参数适配: 1
Simulator object
	Max supported precision: 8
	Min supported precision: 2
	Systolic array size: 2 -inputs x 2 -outputs
	Wbuf size: 65,536 Bytes
	Ibuf size: 32,768 Bytes
	Obuf size: 16,384 Bytes
Double buffering enabled. Sizes of SRAM are halved

**************************************************
Energy costs for BitFusion
Core dynamic energy : 1.955 pJ/cycle (for entire systolic array)
WBUF Read energy    : 0.312 pJ/bit
WBUF Write energy   : 0.263 pJ/bit
IBUF Read energy    : 0.312 pJ/bit
IBUF Write energy   : 0.263 pJ/bit
OBUF Read energy    : 0.200 pJ/bit
OBUF Write energy   : 0.174 pJ/bit



### Generate BitFusion numbers

In [4]:
# CSV 表头
sim_sweep_columns = ['N', 'M',
        'Max Precision (bits)', 'Min Precision (bits)',
        'Network', 'Layer',
        'Cycles', 'Memory wait cycles',
        'WBUF Read', 'WBUF Write',
        'OBUF Read', 'OBUF Write',
        'IBUF Read', 'IBUF Write',
        'DRAM Read', 'DRAM Write',
        'Bandwidth (bits/cycle)',
        'WBUF Size (bits)', 'OBUF Size (bits)', 'IBUF Size (bits)',
        'Batch size']

# bf_e_sim_sweep_csv = os.path.join(results_dir, 'bitfusion-eyeriss-sim-sweep.csv')
bf_e_sim_sweep_csv = os.path.join(results_dir, 'bitfusion_fc_2_2_test-sim-sweep.csv')


#对文件的存在进行判断
# if os.path.exists(bf_e_sim_sweep_csv):
#     bf_e_sim_sweep_df = pandas.read_csv(bf_e_sim_sweep_csv)
# else:
#     #0行21列 空表
#     bf_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)
# print('Got BitFusion Eyeriss, Numbers')

# 直接生成新表
bf_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)



print('batch size : ', batch_size)
bf_e_results = check_pandas_or_run(bf_e_sim, bf_e_sim_sweep_df,
        bf_e_sim_sweep_csv, batch_size=batch_size)

# 'groupby()'方法对 'Network' 列进行分组，并使用 agg() 方法对每个组进行聚合操作，将聚合结果求和。
bf_e_results = bf_e_results.groupby('Network', as_index=False).agg(np.sum)

#面积区域的放置存在问题,仿真的参数表需要重新设置

# area_stats = bf_e_sim.get_area()

# print('area_stats',area_stats)

#此时已经得到了Bit Fusion的 能耗+Benchmarks的次数+面积

INFO:bitfusion.src.sweep.sweep.Simulator:Simulating Benchmark: AlexNet
INFO:bitfusion.src.sweep.sweep.Simulator:N x M = 2 x 2
INFO:bitfusion.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:bitfusion.src.sweep.sweep.Simulator:Min Precision (bits): 2
INFO:bitfusion.src.sweep.sweep.Simulator:Batch size: 1
INFO:bitfusion.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 192


batch size :  1
./conf.ini
{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict {'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict_obuf {'size (bytes)': 8192.0, 'block size (bytes)': 4.0, 'read-write port': 0}

NM_MAX_MIN_参数适配: 1
best_instructions **************************************************
OW/ow: Range 55, stride {'wgt': 0, 'act': 4, 'out': 1}
 | OH/oh: Range 55, stride {'wgt': 0, 'act': 908, 'out': 55}
 |  | Out RD: addr 0, size 1536, stride [1, 55, 6050], level: 2
 |  | Act RD: addr 0, size 976, stride [4, 908, 0], level: 2
 |  | OC/oc: Range 24, stride {'wgt': 726, 'act': 0, 'out': 6050}
 |  |  | Wgt RD: addr 0, size 2912, stride [0, 0, 726], level: 3
 |  | Out WR: addr 0, size 1536, stride [1, 55, 6050], level: 2
**************************************************

best_tiling {'B/b': (1, 1), 'OW/ow': (55, 1), 'OH/oh': (55, 1), 'IC/ic': (1, 3), 'OC/oc': (24, 2)}
best_order ('B/b', 'OW/ow', 'OH/oh', 'IC/ic', 'OC/

INFO:bitfusion.src.sweep.sweep.Simulator:Simulating Benchmark: SVHN
INFO:bitfusion.src.sweep.sweep.Simulator:N x M = 2 x 2
INFO:bitfusion.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:bitfusion.src.sweep.sweep.Simulator:Min Precision (bits): 2
INFO:bitfusion.src.sweep.sweep.Simulator:Batch size: 1
INFO:bitfusion.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 192


best_instructions **************************************************
IC/ic: Range 64, stride {'wgt': 64, 'act': 64, 'out': 0}
 | Act RD: addr 0, size 512, stride [64, 0], level: 1
 | OC/oc: Range 500, stride {'wgt': 8192, 'act': 0, 'out': 2}
 |  | Out RD: addr 0, size 64, stride [0, 2], level: 2
 |  | Wgt RD: addr 0, size 1024, stride [64, 8192], level: 2
 |  | Out WR: addr 0, size 64, stride [0, 2], level: 2
**************************************************

best_tiling {'B/b': (1, 1), 'OW/ow': (1, 1), 'OH/oh': (1, 1), 'IC/ic': (64, 64), 'OC/oc': (500, 2)}
best_order ('B/b', 'OW/ow', 'OH/oh', 'IC/ic', 'OC/oc')
cycle 1024510
num_tiles 32000
stats.mem_stall_cycles 510
stats.compute_cycles 1024000
acc_cycle 32.0
test: 64 2 1 1 1 1 1 8 8 False
Total Cycles: 1,024,510
{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict {'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict_obuf {'size (bytes)': 8192.0, 'block size (bytes)': 4.0,

INFO:bitfusion.src.sweep.sweep.Simulator:Simulating Benchmark: CIFAR10
INFO:bitfusion.src.sweep.sweep.Simulator:N x M = 2 x 2
INFO:bitfusion.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:bitfusion.src.sweep.sweep.Simulator:Min Precision (bits): 2
INFO:bitfusion.src.sweep.sweep.Simulator:Batch size: 1
INFO:bitfusion.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 192


...

best_instructions **************************************************
IC/ic: Range 32, stride {'wgt': 256, 'act': 256, 'out': 0}
 | Act RD: addr 0, size 8192, stride [256, 32768, 0], level: 1
 | B/b: Range 4, stride {'wgt': 0, 'act': 32768, 'out': 4096}
 |  | OC/oc: Range 512, stride {'wgt': 16384, 'act': 0, 'out': 2}
 |  |  | Out RD: addr 0, size 256, stride [0, 4096, 2], level: 3
 |  |  | Wgt RD: addr 0, size 1024, stride [256, 0, 16384], level: 3
 |  |  | Out WR: addr 0, size 256, stride [0, 4096, 2], level: 3
**************************************************

best_tiling {'B/b': (4, 4), 'OW/ow': (1, 1), 'OH/oh': (1, 1), 'IC/ic': (32, 256), 'OC/oc': (512, 2)}
best_order ('OH/oh', 'OW/ow', 'IC/ic', 'B/b', 'OC/oc')
cycle 2097204
num_tiles 65536
stats.mem_stall_cycles 52
stats.compute_cycles 2097152
acc_cycle 32.0
test: 256 2 1 1 4 1 1 2 2 False
Total Cycles: 2,097,204
{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict {'size (bytes)': 16384.0, 'block s

INFO:bitfusion.src.sweep.sweep.Simulator:Simulating Benchmark: RNN
INFO:bitfusion.src.sweep.sweep.Simulator:N x M = 2 x 2
INFO:bitfusion.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:bitfusion.src.sweep.sweep.Simulator:Min Precision (bits): 2
INFO:bitfusion.src.sweep.sweep.Simulator:Batch size: 1
INFO:bitfusion.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 192


best_instructions **************************************************
B/b: Range 2, stride {'wgt': 0, 'act': 331776, 'out': 200704}
 | OW/ow: Range 7, stride {'wgt': 0, 'act': 1, 'out': 1}
 |  | OH/oh: Range 7, stride {'wgt': 0, 'act': 9, 'out': 7}
 |  |  | Act RD: addr 0, size 10240, stride [331776, 1, 9, 1296, 0], level: 3
 |  |  | IC/ic: Range 32, stride {'wgt': 144, 'act': 1296, 'out': 0}
 |  |  |  | OC/oc: Range 256, stride {'wgt': 9216, 'act': 0, 'out': 98}
 |  |  |  |  | Out RD: addr 0, size 512, stride [200704, 1, 7, 0, 98], level: 5
 |  |  |  |  | Wgt RD: addr 0, size 1152, stride [0, 0, 0, 144, 9216], level: 5
 |  |  |  |  | Out WR: addr 0, size 512, stride [200704, 1, 7, 0, 98], level: 5
**************************************************

best_tiling {'B/b': (2, 8), 'OW/ow': (7, 1), 'OH/oh': (7, 1), 'IC/ic': (32, 16), 'OC/oc': (256, 2)}
best_order ('B/b', 'OW/ow', 'OH/oh', 'IC/ic', 'OC/oc')
Total Cycles: 115,605,540
{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-w

INFO:bitfusion.src.sweep.sweep.Simulator:Simulating Benchmark: LSTM
INFO:bitfusion.src.sweep.sweep.Simulator:N x M = 2 x 2
INFO:bitfusion.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:bitfusion.src.sweep.sweep.Simulator:Min Precision (bits): 2
INFO:bitfusion.src.sweep.sweep.Simulator:Batch size: 1
INFO:bitfusion.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 192


best_instructions **************************************************
IC/ic: Range 128, stride {'wgt': 32, 'act': 32, 'out': 0}
 | Act RD: addr 0, size 2048, stride [32, 8192, 0], level: 1
 | B/b: Range 8, stride {'wgt': 0, 'act': 8192, 'out': 8192}
 |  | OC/oc: Range 2048, stride {'wgt': 8192, 'act': 0, 'out': 2}
 |  |  | Out RD: addr 0, size 128, stride [0, 8192, 2], level: 3
 |  |  | Wgt RD: addr 0, size 256, stride [32, 0, 8192], level: 3
 |  |  | Out WR: addr 0, size 128, stride [0, 8192, 2], level: 3
**************************************************

best_tiling {'B/b': (8, 2), 'OW/ow': (1, 1), 'OH/oh': (1, 1), 'IC/ic': (128, 32), 'OC/oc': (2048, 2)}
best_order ('OH/oh', 'OW/ow', 'IC/ic', 'B/b', 'OC/oc')
cycle 16777230
num_tiles 2097152
stats.mem_stall_cycles 14
stats.compute_cycles 16777216
acc_cycle 8.0
test: 32 2 1 1 2 1 1 4 4 False
Total Cycles: 16,777,230
{'size (bytes)': 16384.0, 'block size (bytes)': 4.0, 'read-write port': 0}
cfg_dict {'size (bytes)': 16384.0, 'block size

### Get Eyeriss numbers

In [ ]:
def get_eyeriss_energy(df):

    eyeriss_energy_per_mac = 16 * 0.2 * 1.e-3 #energy in nJ
    # python2 弊病
    # eyeriss_energy_alu = float(df['ALU'])
    # eyeriss_energy_dram = float(df['DRAM']) * 0.15 #Scaling due to technology
    # eyeriss_energy_buffer = float(df['Buffer'])
    # eyeriss_energy_array = float(df['Array'])
    # eyeriss_energy_rf = float(df['RF'])

    eyeriss_energy_alu = pandas.to_numeric(df['ALU'],errors='coerce').astype(float).item()
    eyeriss_energy_dram = pandas.to_numeric(df['DRAM'],errors='coerce').astype(float).item() * 0.15 #Scaling due to technology
    eyeriss_energy_buffer = pandas.to_numeric(df['Buffer'],errors='coerce').astype(float).item()
    eyeriss_energy_array = pandas.to_numeric(df['Array'],errors='coerce').astype(float).item()
    eyeriss_energy_rf = pandas.to_numeric(df['RF'],errors='coerce').astype(float).item()


    eyeriss_energy = eyeriss_energy_alu + eyeriss_energy_dram + eyeriss_energy_buffer + eyeriss_energy_array + eyeriss_energy_rf
    eyeriss_energy *= eyeriss_energy_per_mac
    return eyeriss_energy

def get_eyeriss_energy_breakdown(df):
    eyeriss_energy_per_mac = 16 * 0.2 * 1.e-3 #energy in nJ
    
    # eyeriss_energy_alu = float(df['ALU'])
    # eyeriss_energy_dram = float(df['DRAM'])
    # eyeriss_energy_buffer = float(df['Buffer'])
    # eyeriss_energy_array = float(df['Array'])
    # eyeriss_energy_rf = float(df['RF'])

    eyeriss_energy_alu = pandas.to_numeric(df['ALU'],errors='coerce').astype(float).item()
    eyeriss_energy_dram = pandas.to_numeric(df['DRAM'],errors='coerce').astype(float).item()
    eyeriss_energy_buffer = pandas.to_numeric(df['Buffer'],errors='coerce').astype(float).item()
    eyeriss_energy_array = pandas.to_numeric(df['Array'],errors='coerce').astype(float).item()
    eyeriss_energy_rf = pandas.to_numeric(df['RF'],errors='coerce').astype(float).item()




    eyeriss_energy = [eyeriss_energy_alu+eyeriss_energy_array, eyeriss_energy_buffer, eyeriss_energy_rf, eyeriss_energy_dram]
    eyeriss_energy = [x * eyeriss_energy_per_mac for x in eyeriss_energy]
    return eyeriss_energy

def df_to_stats(df):
    stats = Stats()
    # stats.total_cycles = float(df['Cycles'])
    # stats.mem_stall_cycles = float(df['Memory wait cycles'])
    # stats.reads['act'] = float(df['IBUF Read'])
    # stats.reads['out'] = float(df['OBUF Read'])
    # stats.reads['wgt'] = float(df['WBUF Read'])
    # stats.reads['dram'] = float(df['DRAM Read'])
    # stats.writes['act'] = float(df['IBUF Write'])
    # stats.writes['out'] = float(df['OBUF Write'])
    # stats.writes['wgt'] = float(df['WBUF Write'])
    # stats.writes['dram'] = float(df['DRAM Write'])

    stats.total_cycles = pandas.to_numeric(df['Cycles'],errors='coerce').astype(float).item()
    stats.mem_stall_cycles = pandas.to_numeric(df['Memory wait cycles'],errors='coerce').astype(float).item()
    stats.reads['act'] = pandas.to_numeric(df['IBUF Read'],errors='coerce').astype(float).item()
    stats.reads['out'] = pandas.to_numeric(df['OBUF Read'],errors='coerce').astype(float).item()
    stats.reads['wgt'] = pandas.to_numeric(df['WBUF Read'],errors='coerce').astype(float).item()
    stats.reads['dram'] = pandas.to_numeric(df['DRAM Read'],errors='coerce').astype(float).item()
    stats.writes['act'] = pandas.to_numeric(df['IBUF Write'],errors='coerce').astype(float).item()
    stats.writes['out'] = pandas.to_numeric(df['OBUF Write'],errors='coerce').astype(float).item()
    stats.writes['wgt'] = pandas.to_numeric(df['WBUF Write'],errors='coerce').astype(float).item()
    stats.writes['dram'] = pandas.to_numeric(df['DRAM Write'],errors='coerce').astype(float).item()




    return stats





In [5]:


# Eyeriss-Simulator
# 这个测试的csv得换
# eyeriss_results.csv 给出了具体某一个网络某一层的
# ALU DRAM Buffer Array RF time(ms)
# 横向对比模块
eyeriss_data = pandas.read_csv(os.path.join(results_dir, 'eyeriss_results.csv'))

eyeriss_data_bench = eyeriss_data.groupby('Benchmark', as_index=False).agg(np.sum)

eyeriss_data_bench['Platform'] = 'Eyeriss (16-bit)'
print(eyeriss_data_bench)

# bench='AlexNet'
# lookup_dict = {'Benchmark': bench}
# eyeriss_cycles = lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict)['time(ms)'].astype(float)[0]

# print(eyeriss_cycles[0])




   Benchmark           ALU          DRAM       Buffer         Array  \
0    AlexNet  1.160814e+10  2.296829e+10  922167552.0  1.421881e+09   
1    CIFAR10  9.875128e+09  1.339701e+10  439659072.0  1.229060e+09   
2       LSTM  2.073600e+08  2.638080e+09  113356800.0  5.729280e+07   
3    LeNet-5  2.572452e+08  8.872320e+08   34321536.0  5.291789e+07   
4  RESNET-18  1.803446e+10  3.276046e+10  982813824.0  1.695979e+09   
5        RNN  2.684355e+08  3.394765e+09  101842944.0  6.684672e+07   
6       SVHN  2.530116e+09  4.453114e+09  152467776.0  3.592597e+08   
7      VGG-7  5.095817e+09  9.484262e+09  360025344.0  9.507597e+08   

             RF  time(ms)          Platform  
0  4.677529e+10  70286336  Eyeriss (16-bit)  
1  3.995389e+10  44153856  Eyeriss (16-bit)  
2  8.404992e+08    820800  Eyeriss (16-bit)  
3  1.055392e+09   1117696  Eyeriss (16-bit)  
4  7.260026e+10  95485952  Eyeriss (16-bit)  
5  1.073742e+09   1048576  Eyeriss (16-bit)  
6  1.023410e+10  11296768  Eyeriss (16

In [6]:

print('BitFusion-Eyeriss comparison')
eyeriss_area = 3.5*3.5*45*45/65./65.
print('Area budget = {}'.format(eyeriss_area))


print(area_stats)
if abs(sum(area_stats)-eyeriss_area)/eyeriss_area > 0.1:
    print('Warning: BitFusion Area is outside 10% of eyeriss')
print('total_area = {}, budget = {}'.format(sum(area_stats), eyeriss_area))
bf_e_area = sum(area_stats)

baseline_data = []

# 由于上述csv数据没有benchmarks的数据，所以用test代替
# 得找到适配benchmack的数据再跑一遍
# test = ['AlexNet','LSTM']

for bench in benchmarks.benchlist:

# for bench in test:
    print('bench:',bench)
    lookup_dict = {'Benchmark': bench}

    # eyeriss_cycles = float(lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict)['time(ms)'])
    eyeriss_cycles = lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict)['time(ms)'].astype(float).item()
    
    print('eyeriss_cycles',eyeriss_cycles)

    eyeriss_time = eyeriss_cycles / 500.e3 / 16


    eyeriss_energy = get_eyeriss_energy(lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict))
    eyeriss_power = eyeriss_energy / eyeriss_time * 1.e-9

    eyeriss_speedup = eyeriss_time / eyeriss_time
    eyeriss_energy_efficiency = eyeriss_energy / eyeriss_energy

    eyeriss_ppa = eyeriss_speedup / eyeriss_area / (eyeriss_speedup / eyeriss_area)
    eyeriss_ppw = eyeriss_speedup / eyeriss_power / (eyeriss_speedup / eyeriss_power)
    
    bf_e_stats = df_to_stats(bf_e_results.loc[bf_e_results['Network'] == bench])


    print('bbc',bf_e_stats.total_cycles)
    bf_e_cycles = bf_e_stats.total_cycles * (batch_size / 16.)

    bf_e_time = bf_e_cycles / 500.e3 / 16

    bf_e_energy = bf_e_stats.get_energy(bf_e_sim.get_energy_cost()) * (batch_size / 16.)
    bf_e_power = bf_e_energy / bf_e_time * 1.e-9

    bf_e_speedup = eyeriss_time / bf_e_time

    bf_e_energy_efficiency = eyeriss_energy / bf_e_energy

    bf_e_ppa = bf_e_speedup / bf_e_area / (eyeriss_speedup / eyeriss_area)
    bf_e_ppw = bf_e_speedup / bf_e_power / (eyeriss_speedup / eyeriss_power)

    baseline_data.append(['Performance', bench, bf_e_speedup])
    baseline_data.append(['Energy reduction', bench, bf_e_energy_efficiency])
    baseline_data.append(['Performance-per-Watt', bench, bf_e_ppw])
    baseline_data.append(['Performance-per-Area', bench, bf_e_ppa])
    
    print('*'*50)
    print('Benchmark: {}'.format(bench))
    
    # 二者在时间上对比
    print('Eyeriss time: {} ms'.format(eyeriss_time))
    print('BitFusion time: {} ms'.format(bf_e_time))
   
    # 二者在能耗上对比
    print('Eyeriss power: {} mWatt'.format(eyeriss_power*1.e3*16))
    print('BitFusion power: {} mWatt'.format(bf_e_power*1.e3*16))
    print('*'*50)
    
eyeriss_comparison_df = pandas.DataFrame(baseline_data, columns=['Metric', 'Network', 'Value'])



BitFusion-Eyeriss comparison
Area budget = 5.871301775147929


NameError: name 'area_stats' is not defined

In [7]:
print(eyeriss_data_bench)
eyeriss_data_bench['Platform'] = 'Eyeriss (16-bit)'



print('BitFusion-Eyeriss comparison')
eyeriss_area = 3.5*3.5*45*45/65./65.
print('Area budget = {}'.format(eyeriss_area))


print(area_stats)
if abs(sum(area_stats)-eyeriss_area)/eyeriss_area > 0.1:
    print('Warning: BitFusion Area is outside 10% of eyeriss')
print('total_area = {}, budget = {}'.format(sum(area_stats), eyeriss_area))
bf_e_area = sum(area_stats)

baseline_data = []
for bench in benchmarks.benchlist:

    lookup_dict = {'Benchmark': bench}

    # eyeriss_cycles = float(lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict)['time(ms)'])
    eyeriss_cycles = lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict)['time(ms)'].astype(float).item()
    
    # print('eyeriss_cycles',eyeriss_cycles)

    eyeriss_time = eyeriss_cycles / 500.e3 / 16


    eyeriss_energy = get_eyeriss_energy(lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict))
    eyeriss_power = eyeriss_energy / eyeriss_time * 1.e-9

    eyeriss_speedup = eyeriss_time / eyeriss_time
    eyeriss_energy_efficiency = eyeriss_energy / eyeriss_energy

    eyeriss_ppa = eyeriss_speedup / eyeriss_area / (eyeriss_speedup / eyeriss_area)
    eyeriss_ppw = eyeriss_speedup / eyeriss_power / (eyeriss_speedup / eyeriss_power)
    
    bf_e_stats = df_to_stats(bf_e_results.loc[bf_e_results['Network'] == bench])
    bf_e_cycles = bf_e_stats.total_cycles * (batch_size / 16.)
    bf_e_time = bf_e_cycles / 500.e3 / 16
    bf_e_energy = bf_e_stats.get_energy(bf_e_sim.get_energy_cost()) * (batch_size / 16.)
    bf_e_power = bf_e_energy / bf_e_time * 1.e-9

    bf_e_speedup = eyeriss_time / bf_e_time
    bf_e_energy_efficiency = eyeriss_energy / bf_e_energy

    bf_e_ppa = bf_e_speedup / bf_e_area / (eyeriss_speedup / eyeriss_area)
    bf_e_ppw = bf_e_speedup / bf_e_power / (eyeriss_speedup / eyeriss_power)

    baseline_data.append(['Performance', bench, bf_e_speedup])
    baseline_data.append(['Energy reduction', bench, bf_e_energy_efficiency])
    baseline_data.append(['Performance-per-Watt', bench, bf_e_ppw])
    baseline_data.append(['Performance-per-Area', bench, bf_e_ppa])
    
    print('*'*50)
    print('Benchmark: {}'.format(bench))

    print('Eyeriss time: {} ms'.format(eyeriss_time))

    print('BitFusion time: {} ms'.format(bf_e_time))    
    print('Eyeriss power: {} mWatt'.format(eyeriss_power*1.e3*16))
    print('BitFusion power: {} mWatt'.format(bf_e_power*1.e3*16))
    print('*'*50)
    
eyeriss_comparison_df = pandas.DataFrame(baseline_data, columns=['Metric', 'Network', 'Value'])



   Benchmark           ALU          DRAM       Buffer         Array  \
0    AlexNet  1.160814e+10  2.296829e+10  922167552.0  1.421881e+09   
1    CIFAR10  9.875128e+09  1.339701e+10  439659072.0  1.229060e+09   
2       LSTM  2.073600e+08  2.638080e+09  113356800.0  5.729280e+07   
3    LeNet-5  2.572452e+08  8.872320e+08   34321536.0  5.291789e+07   
4  RESNET-18  1.803446e+10  3.276046e+10  982813824.0  1.695979e+09   
5        RNN  2.684355e+08  3.394765e+09  101842944.0  6.684672e+07   
6       SVHN  2.530116e+09  4.453114e+09  152467776.0  3.592597e+08   
7      VGG-7  5.095817e+09  9.484262e+09  360025344.0  9.507597e+08   

             RF  time(ms)          Platform  
0  4.677529e+10  70286336  Eyeriss (16-bit)  
1  3.995389e+10  44153856  Eyeriss (16-bit)  
2  8.404992e+08    820800  Eyeriss (16-bit)  
3  1.055392e+09   1117696  Eyeriss (16-bit)  
4  7.260026e+10  95485952  Eyeriss (16-bit)  
5  1.073742e+09   1048576  Eyeriss (16-bit)  
6  1.023410e+10  11296768  Eyeriss (16

NameError: name 'area_stats' is not defined

In [8]:
# latex 安装 https://blog.csdn.net/Baby_of_breath/article/details/125667000
bc = BarChart()
bc.LABEL_BAR = True
bc.BAR_WIDTH = None
bc.YAXIS_MAX = 16
bc.ISTIMES = True
bc.TOP_ROTATE = True
bc.LEGEND_LOCATION = "upper center"
bc.LEGEND_NCOLUMN = 5
bc.MV_LEGEND_OUTSIDE_Y = 1.4
bc.MV_LEGEND_OUTSIDE_X = 0.5
bc.LOG_SCALE = False
bc.FIG_WIDTH = 16
bc.TOP_LABEL_FONTSIZE = 12
bc.LEGEND_FONTSIZE = 16

print(eyeriss_comparison_df)
# print(eyeriss_comparison_df)
print(type(eyeriss_comparison_df))


#画图有问题，暂时存成csv
# eyeriss_comparison_df.to_csv('./table/eyeriss-comparision.csv', index=True)


print('Generating Figure for GPU performance comparison')

bc.yaxis = 'Performance-over-Eyeriss'
bc.BAR_WIDTH = None
legend_list = ['Performance', 'Energy reduction']

fig_name = os.path.join(fig_dir, 'eyeriss-comparision.pdf')

# print('h',fig_name)

bc.plot(fig_name, eyeriss_comparison_df, x_plot='Network', y_plot='Value', bar_plot='Metric',
        bar_list=legend_list, add_geomean=True)


NameError: name 'eyeriss_comparison_df' is not defined

### Energy breakdown for Eyeriss and BitFusion

In [ ]:
breakdown_data = []
for bench in benchmarks.benchlist:
    lookup_dict = {'Benchmark': bench}
    eyeriss_energy = get_eyeriss_energy_breakdown(lookup_pandas_dataframe(eyeriss_data_bench, lookup_dict))
    total_energy = sum(eyeriss_energy)
    eyeriss_breakdown = [x / float(total_energy) for x in eyeriss_energy]
    
    breakdown_data.append(['Eyeriss', bench] + eyeriss_breakdown)
    
    bf_e_stats = df_to_stats(bf_e_results.loc[bf_e_results['Network'] == bench])
    bf_breakdown = bf_e_stats.get_energy_breakdown(bf_e_sim.get_energy_cost())
    total = sum(bf_breakdown)
    bf_breakdown = [x / float(total) for x in bf_breakdown]
    
    breakdown_data.append(['BitFusion', bench] + bf_breakdown)
    
breakdown_df = pandas.DataFrame(breakdown_data, columns = ['Platform', 'Network', 'Compute', 'Register File', 'Buffers', 'DRAM'])

In [ ]:
bc = BarChart()
bc.LABEL_BAR = False
bc.BAR_LEFT_MARGIN = 1
bc.BAR_GAP = 0.1
bc.YAXIS_MAX = 1
bc.ISTIMES = True
bc.TOP_ROTATE = True
bc.LEGEND_LOCATION = "upper center"
bc.LEGEND_NCOLUMN = 5
bc.MV_LEGEND_OUTSIDE_Y = 1.2
bc.MV_LEGEND_OUTSIDE_X = 0.5
bc.LOG_SCALE = False
bc.FIG_WIDTH = 16
bc.TOP_LABEL_FONTSIZE = 12
bc.LEGEND_FONTSIZE = 16

print(eyeriss_comparison_df)


# print('Generating Figure for GPU performance comparison')
bc.yaxis = 'Energy breakdown'
bc.BAR_WIDTH = None
legend_list = ['Eyeriss', 'BitFusion']
fig_name = os.path.join(fig_dir, 'bitfusion-eyeriss-energy-breakdown.pdf')
bc.plot(fig_name, breakdown_df, x_plot='Network', y_plot=['Compute', 'Register File', 'Buffers', 'DRAM'], bar_plot='Platform',
        bar_list=legend_list)



## GPU numbers

In [ ]:
gpu_data = []
gpu_list = ['TX2-fp32', 'TitanX-fp32', 'TitanX-int8']
config = ConfigParser.ConfigParser()
config.read('results/output.ini')
for bench in benchmarks.benchlist:
    # Titan-X Pascal numbers
    titanx_fp32_sec = '{}-performance-batch-{}'.format('TitanX-fp32', batch_size)    
    titanx_int8_sec = '{}-performance-batch-{}'.format('TitanX-int8', batch_size)
    titanx_fp32_time = config.getfloat(titanx_fp32_sec, bench.lower())
    titanx_int8_time = config.getfloat(titanx_int8_sec, bench.lower())    
    titanx_fp32_fps = batch_size / titanx_fp32_time * 1.e3
    titanx_int8_fps = batch_size / titanx_int8_time * 1.e3
    int8_speedup = titanx_int8_fps/titanx_fp32_fps
    
    gpu_data.append(['TitanX', bench, 'fp32', titanx_fp32_fps, batch_size])
    gpu_data.append(['TitanX', bench, 'int8', titanx_int8_fps, batch_size])
    
    # Jetson-TX2 numbers
    tx2_fp32_sec = '{}-performance-batch-{}'.format('TX2-fp32', batch_size)    
    tx2_int8_sec = '{}-performance-batch-{}'.format('TX2-int8', batch_size)
    tx2_fp32_time = config.getfloat(tx2_fp32_sec, bench.lower())
    try:
        tx2_int8_time = config.getfloat(tx2_int8_sec, bench.lower())
    except:
        # ID4A speedup
        tx2_int8_time = tx2_fp32_time / int8_speedup
    tx2_fp32_fps = batch_size / tx2_fp32_time * 1.e3 * 2
    tx2_int8_fps = batch_size / tx2_int8_time * 1.e3 * 2
    
    gpu_data.append(['TX2', bench, 'fp32', tx2_fp32_fps, batch_size])
    gpu_data.append(['TX2', bench, 'int8', tx2_int8_fps, batch_size])
    
#     print(v100_fp32_time, v100_int8_time)
#     print(titanx_fp32_time, titanx_int8_time)
    
gpu_data = pandas.DataFrame(gpu_data, columns=['Platform', 'Network', 'Precision', 'fps', 'Batch size'])

gpu_fps_df = gpu_data
gpu_fps_df['Name'] = gpu_fps_df['Platform'] + '-' + gpu_fps_df['Precision']
gpu_fps_df = gpu_fps_df[['Name', 'Network', 'fps', 'Batch size']]

print(gpu_fps_df)

## BitFusion numbers

In [ ]:
# Constant for a particular accelerator configuration
accelerator_constants = ['N', 'M',
                         'Max Precision (bits)', 'Min Precision (bits)',
                         'WBUF Size (bits)', 'OBUF Size (bits)', 'IBUF Size (bits)',
                         'Bandwidth (bits/cycle)'
                        ]
# Constant for a particular network run
algorithm_constants = ['Batch size']


# Variables (to be summed) for a particular network run
variable_columns = ['Cycles', 'Memory wait cycles',
                    'WBUF Read', 'WBUF Write',
                    'OBUF Read', 'OBUF Write',
                    'IBUF Read', 'IBUF Write',
                    'DRAM Read', 'DRAM Write',
                   ]

sim_sweep_columns = accelerator_constants + \
                        algorithm_constants + \
                        variable_columns + \
                        ['Network', 'Layer']


# CSV for sweeps
bf_e_sim_sweep_csv = os.path.join(results_dir, 'bitfusion-eyeriss-sim-sweep.csv')
if os.path.exists(bf_e_sim_sweep_csv):
    bf_e_sim_sweep_df = pandas.read_csv(bf_e_sim_sweep_csv)
else:
    bf_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)
# print('Got BitFusion Eyeriss Numbers')
# print(bf_e_sim_sweep_df)

bf_e_results = check_pandas_or_run(bf_e_sim, bf_e_sim_sweep_df,
                                   bf_e_sim_sweep_csv, batch_size=batch_size)
bf_e_results = bf_e_results.groupby(['Network', 'Batch size']+accelerator_constants,
                                    as_index=False).agg(np.sum)
frequency = bf_e_sim.accelerator.frequency #500 MHz
bf_e_results['Time (ms)'] = bf_e_results['Cycles'] * 1.e3 / frequency # in ms
bf_e_results['fps'] = batch_size / bf_e_results['Time (ms)'] * 1.e3


for bench in benchmarks.benchlist:
    graph = benchmarks.get_bench_nn(bench)
    bf_e_datarow = lookup_pandas_dataframe(bf_e_results, {'Network': bench})
    cycles = bf_e_datarow.Cycles
    mem_cycles = bf_e_datarow['Memory wait cycles']
            
    print('*'*50)
    print(bench)
    bf_e_fps = float(bf_e_datarow['fps'])
    print('\tAchieved FPS = {:.2f} frames/sec'.format(bf_e_fps))
    tx2_fps = float(lookup_pandas_dataframe(gpu_fps_df, {'Name': 'TX2-fp32', 'Network': bench})['fps'])
    titanx_fps = float(lookup_pandas_dataframe(gpu_fps_df, {'Name': 'TitanX-int8', 'Network': bench})['fps'])
    
    titanx_speedup = titanx_fps / tx2_fps
    bf_e_speedup = bf_e_fps / tx2_fps
    
    print('\tBitfusion speedup over TX2: {}'.format(bf_e_speedup))
    print('\tTitanX speedup over TX2: {}'.format(titanx_speedup))
    
    print('*'*50)

## Generate performance comparisons

In [ ]:
bf_e_fps_df = bf_e_results[['Network', 'fps']]
bf_e_fps_df['Name'] = 'BitFusion-45nm'

all_fps_df = pandas.concat([bf_e_fps_df, gpu_fps_df])[['Network', 'Name', 'fps']]
print(all_fps_df)
platform_list = ['TX2-fp32', 'TitanX-fp32', 'TitanX-int8', 'BitFusion-45nm']
all_fps_df = all_fps_df[all_fps_df['Name'].isin(platform_list)]
# print(all_fps_df)

## GPU performance comparison figure

In [ ]:
bc = BarChart()
bc.LABEL_BAR = True
bc.BAR_WIDTH = None
bc.YAXIS_MAX = 5
bc.ISTIMES = True
bc.TOP_ROTATE = True
bc.LEGEND_LOCATION = "upper center"
bc.LEGEND_NCOLUMN = 5
bc.MV_LEGEND_OUTSIDE_Y = 1.4
bc.MV_LEGEND_OUTSIDE_X = 0.5
bc.LOG_SCALE = False
bc.FIG_WIDTH = 16
bc.TOP_LABEL_FONTSIZE = 12
bc.LEGEND_FONTSIZE = 16

# print('Generating Figure for GPU performance comparison')
bc.yaxis = 'Performance-over-TX2'
bc.BAR_WIDTH = None
platform_list = ['TX2-fp32', 'TitanX-fp32', 'TitanX-int8', 'BitFusion-45nm']
fig_name = os.path.join(fig_dir, 'performance-gpu.pdf')
bc.plot(fig_name, all_fps_df, x_plot='Network', y_plot='fps', bar_plot='Name',
        bar_list=platform_list, add_geomean=True, baseline='TX2-fp32')

## Get 2x, 4x, and 8x area performance for bitfusion

In [ ]:
bf_fps_df = pandas.DataFrame(columns=['Name', 'Network', 'fps'])
df_list = []
for factor in [1, 2, 4, 8]:
    config_file = 'bf_e_conf.ini'
    verbose = False
    bf_scaled = Simulator(config_file, verbose)
    bf_scaled.accelerator.M *= factor
    bf_scaled.accelerator.sram['act'] *= factor
    bf_scaled.accelerator.sram['wgt'] *= factor
    bf_scaled.accelerator.sram['out'] *= factor
#     if bf_scaled.accelerator.mem_if_width * factor > 1024:
#         bf_scaled.accelerator.mem_if_width = 1024
#     else:
#         bf_scaled.accelerator.mem_if_width = bf_scaled.accelerator.mem_if_width * factor
#     bf_scaled.accelerator.mem_if_width = bf_scaled.accelerator.mem_if_width * factor
    bf_scaled.accelerator.mem_if_width = 4096
    frequency = bf_scaled.accelerator.frequency
#     print(bf_scaled)
    bf_e_scaled_results = check_pandas_or_run(bf_scaled, bf_e_sim_sweep_df,
                                       bf_e_sim_sweep_csv, batch_size=batch_size)
    bf_e_scaled_results = bf_e_scaled_results.groupby(['Network', 'Batch size']+accelerator_constants,
                                    as_index=False).agg(np.sum)
    bf_e_scaled_results['fps'] = batch_size * frequency / (bf_e_scaled_results.Cycles)
    bf_e_scaled_results['Name'] = 'BitFusion ({}x)'.format(factor)
    bf_e_scaled_results = bf_e_scaled_results[['Name', 'Network', 'fps']]
    df_list.append(bf_e_scaled_results)
    
bf_fps_df = pandas.concat(df_list)
all_fps_df = pandas.concat([all_fps_df, bf_fps_df])
print(all_fps_df)

In [ ]:
bc = BarChart()
bc.LABEL_BAR = True
bc.BAR_WIDTH = None
bc.YAXIS_MAX = 16
bc.FIG_WIDTH = 16
bc.TOP_LABEL_FONTSIZE = 12
bc.LEGEND_FONTSIZE = 16
bc.ISTIMES = True
bc.TOP_ROTATE = True
bc.LEGEND_LOCATION = "upper center"
bc.LEGEND_NCOLUMN = 5
bc.MV_LEGEND_OUTSIDE_Y = 1.6
bc.MV_LEGEND_OUTSIDE_X = 0.5
bc.LOG_SCALE = False

# print('Generating Figure for GPU performance comparison')
bc.yaxis = 'Performance'
bc.BAR_WIDTH = None
fig_name = os.path.join(fig_dir, 'performance-eyeriss-comparision.pdf')
platform_list = ['TX2-fp32', 'TitanX-fp32', 'TitanX-int8', 'BitFusion (8x)']
bc.plot(fig_name, all_fps_df, x_plot='Network', y_plot='fps', bar_plot='Name',
        bar_list=platform_list, add_geomean=True, baseline='TX2-fp32')